# Create interactive plots

This notebook is to assist in exploring the results

In [1]:
%matplotlib inline

import sys
import warnings
import odc.geo.xr
import xarray as xr
import branca.colormap as cm
import matplotlib as mpl
from odc.geo.xr import assign_crs

sys.path.append('/g/data/os22/chad_tmp/AusEFlux/src/')
from _utils import round_coords

/g/data/os22/chad_tmp/AusENDVI/env/py310/lib/python3.10/site-packages/dask/config.py:779: FutureWarning: Dask configuration key 'ucx' has been deprecated; please use 'distributed.comm.ucx' instead
  warnings.warn(
/g/data/os22/chad_tmp/AusENDVI/env/py310/lib/python3.10/site-packages/dask/config.py:779: FutureWarning: Dask configuration key 'distributed.scheduler.transition-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/g/data/os22/chad_tmp/AusENDVI/env/py310/lib/python3.10/site-packages/dask/config.py:779: FutureWarning: Dask configuration key 'distributed.comm.recent-messages-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(


## Analysis Parameters


In [3]:
p_average = assign_crs(xr.open_dataset('/g/data/os22/chad_tmp/Aus_phenology/results/mean_phenology_perpixel.nc'), crs='EPSG:4326')
p_trends = assign_crs(xr.open_dataset('/g/data/os22/chad_tmp/Aus_phenology/results/trends_phenology_perpixel.nc'), crs='EPSG:4326')
# p_parcorr = assign_crs(xr.open_dataset('/g/data/os22/chad_tmp/Aus_phenology/results/parcorr_phenology_perpixel.nc'), crs='EPSG:4326')

## Masking

In [4]:
crops = xr.open_dataset('/g/data/os22/chad_tmp/Aus_phenology/data/croplands_5km.nc')['croplands']
crops = xr.where(crops!=1, 0, 1) #all cropping
crops = round_coords(crops)

In [5]:
p_trends = round_coords(p_trends)
p_trends = p_trends.where(crops)

# p_parcorr = round_coords(p_parcorr)
# p_parcorr = p_parcorr.where(crops)

## Long-term average phenology




In [ ]:
var = 'SOS'
cmap='twilight'
vmin, vmax= 0,365
m = p_average[var].odc.explore(
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            cmap=cmap,
            vmin=vmin,
            vmax=vmax
)

cmaps = mpl.colormaps[cmap]
colormap = cm.LinearColormap(cmaps.colors, vmin=vmin,vmax=vmax, caption=var)
m.add_child(colormap)

## Trends

In [7]:
var = 'vTOS'
cmap='BrBG'
vmin, vmax= -0.002,0.002

m = p_trends[var+'_slope'].where(p_trends['vPOS_p_value']<=0.05).odc.explore(
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            cmap=cmap,
            vmin=vmin,
            vmax=vmax
)
m
# cmaps = mpl.colormaps[cmap]
# colormap = cm.LinearColormap(cmaps, vmin=vmin,vmax=vmax, caption=var)
# m.add_child(colormap)

In [ ]:
var = 'IOS'
cmap='PuOr'
vmin, vmax= -1,1

m = p_trends[var+'_slope'].where(p_trends[var+'_p_value']<=0.05).odc.explore(
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            cmap=cmap,
            vmin=vmin,
            vmax=vmax
)
m

## Seasonal component most correlated with IOS

In [ ]:
import numpy as np
import sys
sys.path.append('/g/data/os22/chad_tmp/AusEFlux/src/')
from _prediction import allNaN_arg

import distinctipy
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
max_corr = allNaN_arg(np.abs(p_parcorr.to_array()), dim='variable',stat='max', idx=False)

In [ ]:
categories = list(p_parcorr.data_vars)
colours = distinctipy.get_colors(len(categories), pastel_factor=0.5, rng=2)
cmap = LinearSegmentedColormap.from_list("cwc", colours, N=len(categories))
vmin, vmax= 0,7

m = max_corr.odc.explore(
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            cmap=cmap,
            vmin=vmin,
            vmax=vmax
)
m